In [ ]:
import json
import os
import random
from collections import defaultdict, Counter
from pathlib import Path

print("=" * 80)
print("📦 ACORD Dataset Conversion - Google Drive Format")
print("=" * 80)

📦 ACORD Dataset Conversion - Google Drive Format


In [ ]:
# Define paths - UPDATE THESE based on where you upload files
CORPUS_PATH = "corpus.jsonl"
QUERIES_PATH = "queries.jsonl"
QRELS_PATH = "qrels"

In [ ]:
print("\n📂 Loading ACORD files...")

# Load corpus (contract clauses)
print(f"Loading corpus from: {CORPUS_PATH}")
corpus_dict = {}
with open(CORPUS_PATH, 'r') as f:
    for line in f:
        item = json.loads(line)
        corpus_dict[item['_id']] = item
print(f"✅ Loaded {len(corpus_dict)} clauses")

# Load queries (clause type descriptions)
print(f"Loading queries from: {QUERIES_PATH}")
queries_dict = {}
with open(QUERIES_PATH, 'r') as f:
    for line in f:
        item = json.loads(line)
        queries_dict[item['_id']] = item
print(f"✅ Loaded {len(queries_dict)} queries")

# Show examples
print(f"\n📋 Example Corpus Entry:")
if corpus_dict:
    sample_corpus_id = list(corpus_dict.keys())[0]
    sample_corpus = corpus_dict[sample_corpus_id]
    print(f"   ID: {sample_corpus['_id']}")
    print(f"   Text: {sample_corpus['text'][:200]}...")
    print(f"   Metadata: {sample_corpus.get('metadata', {})}")
else:
    print("   Corpus is empty.")

print(f"\n📋 Example Query:")
if queries_dict:
    sample_query_id = list(queries_dict.keys())[0]
    sample_query = queries_dict[sample_query_id]
    print(f"   ID: {sample_query['_id']}")
    print(f"   Text: {sample_query['text']}")
    print(f"   Metadata: {sample_query.get('metadata', {})}")
else:
    print("   Queries are empty.")


📂 Loading ACORD files...
Loading corpus from: corpus.jsonl
✅ Loaded 3931 clauses
Loading queries from: queries.jsonl
✅ Loaded 114 queries

📋 Example Corpus Entry:
   ID: a5a68dbd19
   Text: In no event may either party sell, disclose, transfer, rent, or license Payment-Eligible User Data to the other party's Named Competitors as listed in EXHIBIT E. Furthermore, Excite@Home may not sell,...
   Metadata: {}

📋 Example Query:
   ID: New York Governing Law
   Text: New York Governing Law
   Metadata: {'category': 'Governing Law', 'parent_query_id': '', 'type': 'cuad1', 'split': 'train'}


In [ ]:
 #CELL 2: Load Qrels (Relevance Judgments)
# ============================================================================

def load_qrels(qrels_folder):
    """
    Load qrels files (TSV format: query-id corpus-id score)
    Returns dict mapping splits to list of (query_id, corpus_id, score) tuples
    """
    qrels_data = {}

    # Look for train, test, validation TSV files
    qrels_files = {
        'train': 'train.tsv',
        'test': 'test.tsv',
        'validation': 'dev.tsv'  # Sometimes called dev.tsv
    }

    for split, filename in qrels_files.items():
        filepath = os.path.join(qrels_folder, filename)
        if not os.path.exists(filepath):
            print(f"⚠️  {filename} not found, skipping...")
            continue

        qrels_data[split] = []
        with open(filepath, 'r') as f:
            # Skip the header row
            next(f)
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 3:
                    query_id = parts[0]
                    corpus_id = parts[1]
                    score = int(parts[2])
                    qrels_data[split].append((query_id, corpus_id, score))

        print(f"✅ Loaded {len(qrels_data[split])} qrels for {split}")

    return qrels_data

print(f"\n📊 Loading qrels (relevance judgments)...")
qrels = load_qrels(QRELS_PATH)

# Show statistics
print(f"\n📈 Qrels Statistics:")
for split, data in qrels.items():
    scores = [score for _, _, score in data]
    print(f"   {split}: {len(data)} pairs")
    print(f"      Score distribution: {Counter(scores)}")


📊 Loading qrels (relevance judgments)...
✅ Loaded 58529 qrels for train
✅ Loaded 61988 qrels for test
⚠️  dev.tsv not found, skipping...

📈 Qrels Statistics:
   train: 58529 pairs
      Score distribution: Counter({0: 56973, 1: 1036, 3: 287, 2: 159, 4: 74})
   test: 61988 pairs
      Score distribution: Counter({0: 60231, 1: 1137, 3: 348, 2: 219, 4: 53})


In [ ]:
# ============================================================================
# CELL 2: Explore ACORD Format
# ============================================================================

print("\n🔍 Exploring ACORD Structure...")
print("="*80)

# ACORD uses BEIR format with 3 components:
# 1. Queries (questions asking for specific clause types)
# 2. Corpus (actual clause texts from contracts)
# 3. Qrels (relevance scores: 0-4 stars, where 0=irrelevant, 4=perfect)

print(f"\n📋 Sample Query:")
if queries_dict:
    sample_query = list(queries_dict.values())[0]
    print(json.dumps(sample_query, indent=2)[:500])
else:
    print("   Queries dictionary is empty.")


print(f"\n📋 Sample Corpus Entry:")
if corpus_dict:
    sample_corpus = list(corpus_dict.values())[0]
    print(json.dumps(sample_corpus, indent=2)[:500])
else:
    print("   Corpus dictionary is empty.")


print(f"\n📊 ACORD Statistics:")
print(f"   Queries: {len(queries_dict)} expert-written")
print(f"   Clauses: {len(corpus_dict)} from contracts")
total_pairs = sum(len(qrels[split]) for split in qrels) # Summing up pairs from all splits in qrels
print(f"   Query-Clause Pairs: {total_pairs}")
print(f"   Ratings: 0-4 stars (by lawyers)")


🔍 Exploring ACORD Structure...

📋 Sample Query:
{
  "_id": "New York Governing Law",
  "text": "New York Governing Law",
  "metadata": {
    "category": "Governing Law",
    "parent_query_id": "",
    "type": "cuad1",
    "split": "train"
  }
}

📋 Sample Corpus Entry:
{
  "_id": "a5a68dbd19",
  "text": "In no event may either party sell, disclose, transfer, rent, or license Payment-Eligible User Data to the other party's Named Competitors as listed in EXHIBIT E. Furthermore, Excite@Home may not sell, disclose, transfer, rent, or license Shopping Category Data or Superset Data to Data Restricted Named Companies as specified in EXHIBIT I. Not more than once per quarter, Application Provider may update the list of Application Provider Data Restricted Named Compa

📊 ACORD Statistics:
   Queries: 114 expert-written
   Clauses: 3931 from contracts
   Query-Clause Pairs: 120517
   Ratings: 0-4 stars (by lawyers)


In [ ]:
# ============================================================================
# CELL 3: Convert ACORD to Training Format
# ============================================================================

import json
from collections import defaultdict

print("\n🔄 Converting ACORD to instruction format...")
print("="*80)

def convert_acord_to_instruction_format(qrels_data, queries_dict, corpus_dict, min_relevance=3):
    """
    Convert ACORD BEIR format to instruction format for fine-tuning.

    Args:
        qrels_data: Dict of split -> list of (query_id, corpus_id, score) tuples
        queries_dict: Dict of query_id -> query object
        corpus_dict: Dict of corpus_id -> corpus object
        min_relevance: Minimum relevance score (0-4). Default 3 = 4-5 star clauses.

    Returns:
        Dict of split -> list of training samples in instruction format
    """

    converted_data = {}

    for split, qrel_list in qrels_data.items():
        samples = []

        for query_id, corpus_id, score in qrel_list:
            # Filter by minimum score (3-4 stars = good/excellent)
            if score < min_relevance:
                continue

            # Get query and corpus objects
            query = queries_dict.get(query_id)
            corpus_item = corpus_dict.get(corpus_id)

            if not query or not corpus_item:
                continue

            # Extract data
            query_text = query['text']
            clause_text = corpus_item['text']
            category = query.get('metadata', {}).get('category', 'General')

            # Create instruction format sample
            sample = {
                'instruction': f'Extract the exact {category} clause that matches this requirement',
                'input': f"Requirement: {query_text}\n\nContract Text: {clause_text}",
                'output': clause_text,
                'clause_type': category,
                'relevance_score': score,
                'metadata': {
                    'query': query_text,
                    'split': split
                }
            }

            samples.append(sample)

        converted_data[split] = samples

    return converted_data

# Convert data with min_relevance=3 (only 3-4 star clauses)
acord_data = convert_acord_to_instruction_format(qrels, queries_dict, corpus_dict, min_relevance=3)

print(f"✅ Conversion complete!")
for split, samples in acord_data.items():
    print(f"   {split}: {len(samples)} samples")

# Show example
if 'train' in acord_data and len(acord_data['train']) > 0:
    print(f"\n📝 Example Training Sample:")
    example = acord_data['train'][0]
    print(f"   Instruction: {example['instruction']}")
    print(f"   Input (first 200 chars): {example['input'][:200]}...")
    print(f"   Output (first 200 chars): {example['output'][:200]}...")
    print(f"   Clause Type: {example['clause_type']}")
    print(f"   Relevance: {example['relevance_score']}/4 stars")


🔄 Converting ACORD to instruction format...
✅ Conversion complete!
   train: 347 samples
   test: 391 samples

📝 Example Training Sample:
   Instruction: Extract the exact Limitation of Liability clause that matches this requirement
   Input (first 200 chars): Requirement: Cap on liability without carveouts

Contract Text: 15. LIMITATIONS OF LIABILITY
MORPHO agrees that XIMAGE's total liability to MORPHO for any damages suffered in connection with, or arisi...
   Output (first 200 chars): 15. LIMITATIONS OF LIABILITY
MORPHO agrees that XIMAGE's total liability to MORPHO for any damages suffered in connection with, or arising out of, this Agreement or MORPHO's use of any documentation, ...
   Clause Type: Limitation of Liability
   Relevance: 3/4 stars


In [ ]:
import random

print("\n🔄 Applying data augmentation...")
print("="*80)

def augment_training_data(data, augmentation_factor=2):
    """Augment training data by creating variations."""

    augmented = []

    instruction_templates = [
        'Extract the exact {clause_type} clause that matches this requirement',
        'Copy the {clause_type} clause from this contract that satisfies the following',
        'Find and extract the {clause_type} clause based on this description',
        'Locate the {clause_type} clause in the contract matching',
        'Identify and extract the relevant {clause_type} clause for',
    ]

    for sample in data:
        augmented.append(sample)  # Original

        # Create variations
        for i in range(augmentation_factor - 1):
            template = random.choice(instruction_templates)
            aug_sample = sample.copy()
            aug_sample['instruction'] = template.format(
                clause_type=sample['clause_type']
            )
            augmented.append(aug_sample)

    return augmented

# Check combined train+val size
all_data_size = 0
if 'train' in acord_data:
    all_data_size += len(acord_data['train'])
if 'validation' in acord_data:
    all_data_size += len(acord_data['validation'])

print(f"Current dataset size: {all_data_size} samples")

# FORCE 3x augmentation since dataset is small
augmentation_factor = 3
print(f"⚠️ Dataset is small - applying {augmentation_factor}x augmentation")

if 'train' in acord_data:
    original_size = len(acord_data['train'])
    acord_data['train'] = augment_training_data(acord_data['train'], augmentation_factor)
    print(f"✅ Train: {original_size} → {len(acord_data['train'])} samples")

if 'validation' in acord_data:
    original_size = len(acord_data['validation'])
    acord_data['validation'] = augment_training_data(acord_data['validation'], augmentation_factor)
    print(f"✅ Validation: {original_size} → {len(acord_data['validation'])} samples")

# Test is NOT augmented
print(f"✅ Test: {len(acord_data['test'])} samples (no augmentation)")


🔄 Applying data augmentation...
Current dataset size: 1041 samples
⚠️ Dataset is small - applying 3x augmentation
✅ Train: 884 → 2652 samples
✅ Validation: 157 → 471 samples
✅ Test: 391 samples (no augmentation)


In [ ]:
# ============================================================================
# CELL 5: Create Train/Val/Test Splits
# ============================================================================

from sklearn.model_selection import train_test_split
from collections import Counter

print("\n📊 Creating data splits...")
print("="*80)

# If validation split doesn't exist, create it from train
if 'validation' not in acord_data and 'train' in acord_data:
    print("⚠️ No validation split found, creating from train data...")
    train_data = acord_data['train']
    random.seed(42)
    random.shuffle(train_data)

    # Split: 85% train, 15% val
    train_split, val_split = train_test_split(train_data, test_size=0.15, random_state=42)
    acord_data['train'] = train_split
    acord_data['validation'] = val_split
    print(f"✅ Created validation split from train data")

# Use existing splits
train_split = acord_data.get('train', [])
val_split = acord_data.get('validation', [])
test_split = acord_data.get('test', [])

print(f"✅ Data splits:")
print(f"   Train: {len(train_split)} samples")
print(f"   Validation: {len(val_split)} samples")
print(f"   Test: {len(test_split)} samples")

# Verify clause type distribution
train_types = Counter(s['clause_type'] for s in train_split)
print(f"\n📋 Clause type distribution in training set:")
for clause_type, count in train_types.most_common():
    print(f"   {clause_type}: {count}")


📊 Creating data splits...
✅ Data splits:
   Train: 2652 samples
   Validation: 471 samples
   Test: 391 samples

📋 Clause type distribution in training set:
   Limitation of Liability: 1644
   Indemnification: 513
   Restrictive Covenants: 189
   Term: 123
   Governing Law: 90
   Affirmative Covenants: 48
   IP Ownership/License: 45


In [ ]:
# ============================================================================
# CELL 6: Save Converted Data
# ============================================================================

import json
import os

print("\n💾 Saving converted ACORD data...")
print("="*80)

# Create output directory
output_dir = "/kaggle/working/acord_data"
os.makedirs(output_dir, exist_ok=True)

# Save splits
splits = {
    'train': train_split,
    'val': val_split,
    'test': test_split
}

for split_name, split_data in splits.items():
    output_path = f"{output_dir}/{split_name}.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(split_data, f, indent=2, ensure_ascii=False)

    file_size_mb = os.path.getsize(output_path) / 1024 / 1024
    print(f"✅ Saved {split_name}.json - {len(split_data)} samples ({file_size_mb:.2f} MB)")

# Save metadata
metadata = {
    'dataset': 'ACORD (Atticus Clause Retrieval Dataset)',
    'source': 'theatticusproject/acord (local files)',
    'conversion_date': '2025-10-30',
    'min_relevance': 3,
    'augmentation_factor': augmentation_factor,
    'splits': {
        'train': len(train_split),
        'validation': len(val_split),
        'test': len(test_split)
    },
    'clause_types': dict(train_types)
}

with open(f"{output_dir}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ All data saved to: {output_dir}")
print(f"   Files: train.json, val.json, test.json, metadata.json")



💾 Saving converted ACORD data...
✅ Saved train.json - 2652 samples (9.25 MB)
✅ Saved val.json - 471 samples (1.68 MB)
✅ Saved test.json - 391 samples (1.42 MB)

✅ All data saved to: /kaggle/working/acord_data
   Files: train.json, val.json, test.json, metadata.json


In [ ]:
# ============================================================================
# CELL 7: Data Quality Validation
# ============================================================================

print("\n🔍 Validating data quality...")
print("="*80)

def validate_training_data(data, split_name='train'):
    """Validate that ACORD data is correctly formatted and high quality."""

    issues = []

    for i, sample in enumerate(data[:100]):  # Check first 100
        # Check required fields
        if not sample.get('instruction'):
            issues.append(f"Sample {i}: Missing instruction")
        if not sample.get('input'):
            issues.append(f"Sample {i}: Missing input")
        if not sample.get('output'):
            issues.append(f"Sample {i}: Missing output")

        # Check that output is substring of input (for extraction task)
        if sample.get('output') and sample.get('input'):
            # For ACORD, output should be in input (same clause)
            # This is KEY difference from CUAD where they didn't match!
            if sample['output'] not in sample['input']:
                issues.append(f"Sample {i}: Output not found in input")

        # Check length
        if len(sample.get('output', '')) < 20:
            issues.append(f"Sample {i}: Output too short ({len(sample['output'])} chars)")

    if issues:
        print(f"⚠️ Found {len(issues)} potential issues in {split_name}:")
        for issue in issues[:10]:  # Show first 10
            print(f"   {issue}")
    else:
        print(f"✅ {split_name} data quality: EXCELLENT")

    # Statistics
    avg_input_len = sum(len(s['input']) for s in data) / len(data)
    avg_output_len = sum(len(s['output']) for s in data) / len(data)

    print(f"\n📊 {split_name} Statistics:")
    print(f"   Average input length: {avg_input_len:.0f} characters")
    print(f"   Average output length: {avg_output_len:.0f} characters")
    print(f"   Input/Output ratio: {avg_input_len/avg_output_len:.2f}x")

    return len(issues) == 0

# Validate all splits
train_valid = validate_training_data(train_split, 'Train')
val_valid = validate_training_data(val_split, 'Validation')
test_valid = validate_training_data(test_split, 'Test')

if train_valid and val_valid and test_valid:
    print(f"\n🎉 ALL DATA VALIDATED - READY FOR TRAINING!")
    print(f"\n📝 Next Steps:")
    print(f"   1. Use these paths in your training code:")
    print(f"      TRAIN_PATH = '/kaggle/working/acord_data/train.json'")
    print(f"      VAL_PATH = '/kaggle/working/acord_data/val.json'")
    print(f"   2. Rest of your training code stays THE SAME!")
    print(f"   3. Expected results: 60-80% similarity (vs CUAD's 24%)")
else:
    print(f"\n⚠️ Please review and fix issues before training")


🔍 Validating data quality...
✅ Train data quality: EXCELLENT

📊 Train Statistics:
   Average input length: 1701 characters
   Average output length: 1621 characters
   Input/Output ratio: 1.05x
✅ Validation data quality: EXCELLENT

📊 Validation Statistics:
   Average input length: 1740 characters
   Average output length: 1659 characters
   Input/Output ratio: 1.05x
✅ Test data quality: EXCELLENT

📊 Test Statistics:
   Average input length: 1773 characters
   Average output length: 1701 characters
   Input/Output ratio: 1.04x

🎉 ALL DATA VALIDATED - READY FOR TRAINING!

📝 Next Steps:
   1. Use these paths in your training code:
      TRAIN_PATH = '/kaggle/working/acord_data/train.json'
      VAL_PATH = '/kaggle/working/acord_data/val.json'
   2. Rest of your training code stays THE SAME!
   3. Expected results: 60-80% similarity (vs CUAD's 24%)


# From here

In [ ]:
# Install datasets library
import subprocess
import sys

subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'datasets'])

print("📥 Downloading ACORD dataset...")
from datasets import load_dataset

# Download ACORD
acord_data = load_dataset("theatticusproject/acord")

print(f"✅ ACORD loaded!")
print(f"   Train split: {len(acord_data['train'])} samples")
print(f"   Test split: {len(acord_data['test'])} samples")
print(f"   Validation split: {len(acord_data['validation'])} samples")


In [ ]:
import json

print("🔍 Exploring ACORD Structure...")
print("="*80)

# ACORD uses BEIR format with 3 components:
# 1. Queries (questions asking for specific clause types)
# 2. Corpus (actual clause texts from contracts)
# 3. Qrels (relevance scores: 0-4 stars, where 0=irrelevant, 4=perfect)

# Load queries
queries_file = acord_data['train']  # Example access
print(f"\n📋 Sample Query:")
print(json.dumps(queries_file, indent=2)[:500])

# Show statistics
print(f"\n📊 ACORD Statistics:")
print(f"   Queries: 114 expert-written")
print(f"   Clauses: 3000+ from 450 contracts")
print(f"   Query-Clause Pairs: 126,000+")
print(f"   Ratings: 1-5 stars (by lawyers)")
print(f"   Categories: 9 complex clause types")


In [ ]:
import json
from collections import defaultdict

print("🔄 Converting ACORD to instruction format...")
print("="*80)

# ACORD format conversion strategy:
# - Each query-clause pair with rating >= 3 (good/excellent) becomes a training sample
# - Query text → instruction
# - Clause text → both input and output (teach model to extract/copy)
# - Include relevance score for future weighted training

def convert_acord_to_instruction_format(acord_dataset, min_relevance=3):
    """
    Convert ACORD BEIR format to instruction format for fine-tuning.

    Args:
        acord_dataset: ACORD dataset from HuggingFace
        min_relevance: Minimum relevance score (0-4). Default 3 = 4-5 star clauses.

    Returns:
        List of training samples in instruction format
    """

    training_data = []

    # Access ACORD components
    # Note: Actual structure depends on how ACORD is formatted
    # You may need to adjust based on actual data structure

    for split_name in ['train', 'validation']:
        if split_name not in acord_dataset:
            continue

        split_data = acord_dataset[split_name]

        for item in split_data:
            # Extract query and clause
            # Adjust field names based on actual ACORD structure
            query_text = item.get('query', item.get('question', ''))
            clause_text = item.get('clause', item.get('text', item.get('context', '')))
            relevance = item.get('score', item.get('relevance', 0))
            category = item.get('category', item.get('clause_type', 'General'))

            # Only include relevant clauses (3-4 stars = 4-5 star rating)
            if relevance >= min_relevance:
                training_data.append({
                    'instruction': f'Extract the exact {category} clause that matches this requirement',
                    'input': f"Requirement: {query_text}\n\nContract Text: {clause_text}",
                    'output': clause_text,
                    'clause_type': category,
                    'relevance_score': relevance,
                    'metadata': {
                        'query': query_text,
                        'split': split_name
                    }
                })

    return training_data

# Convert data
train_data = convert_acord_to_instruction_format(acord_data, min_relevance=3)

print(f"✅ Conversion complete!")
print(f"   Total samples: {len(train_data)}")
print(f"   Min relevance: 3 stars (good/excellent clauses)")

# Show example
print(f"\n📝 Example Training Sample:")
print(f"   Instruction: {train_data['instruction']}")
print(f"   Input (first 200 chars): {train_data['input'][:200]}...")
print(f"   Output (first 200 chars): {train_data['output'][:200]}...")
print(f"   Clause Type: {train_data['clause_type']}")
print(f"   Relevance: {train_data['relevance_score']}/4 stars")



🔄 Converting to instruction format...
✅ Converted 1496 samples for train
✅ Converted 1726 samples for test

📝 Example Training Sample:
   Instruction: Extract the exact Limitation of Liability clause that matches this requirement
   Input (first 200 chars): Requirement: Cap on liability without carveouts

Contract Text: 15. LIMITATIONS OF LIABILITY
MORPHO agrees that XIMAGE's total liability to MORPHO for any damages suffered in connection with, or arisi...
   Output (first 200 chars): 15. LIMITATIONS OF LIABILITY
MORPHO agrees that XIMAGE's total liability to MORPHO for any damages suffered in connection with, or arising out of, this Agreement or MORPHO's use of any documentation, ...
   Clause Type: Limitation of Liability
   Relevance: 3/4


In [ ]:
import random

print("🔄 Applying data augmentation...")
print("="*80)

def augment_training_data(data, augmentation_factor=2):
    """
    Augment training data by creating variations.
    Useful if ACORD dataset is too small (114 queries).
    """

    augmented = []

    # Instruction variations
    instruction_templates = [
        'Extract the exact {clause_type} clause that matches this requirement',
        'Copy the {clause_type} clause from this contract that satisfies the following',
        'Find and extract the {clause_type} clause based on this description',
        'Locate the {clause_type} clause in the contract matching',
        'Identify and extract the relevant {clause_type} clause for',
    ]

    for sample in data:
        # Add original
        augmented.append(sample)

        # Create variations with different instruction templates
        for i in range(augmentation_factor - 1):
            template = random.choice(instruction_templates)
            augmented_sample = sample.copy()
            augmented_sample['instruction'] = template.format(
                clause_type=sample['clause_type']
            )
            augmented.append(augmented_sample)

    return augmented

# Augment if needed
if len(train_data) < 1000:
    print(f"⚠️ Dataset has {len(train_data)} samples - applying 3x augmentation")
    train_data = augment_training_data(train_data, augmentation_factor=3)
    print(f"✅ Augmented to {len(train_data)} samples")
else:
    print(f"✅ Dataset has {len(train_data)} samples - augmentation not needed")



📈 Data Augmentation...
⚠️  Training set has 1496 samples - applying 3x augmentation
✅ Augmented to 4488 samples


In [ ]:
from sklearn.model_selection import train_test_split

print("📊 Creating data splits...")
print("="*80)

# Shuffle data
random.seed(42)
random.shuffle(train_data)

# Split: 70% train, 15% val, 15% test
train_split, temp = train_test_split(train_data, test_size=0.3, random_state=42)
val_split, test_split = train_test_split(temp, test_size=0.5, random_state=42)

print(f"✅ Data splits created:")
print(f"   Train: {len(train_split)} samples (70%)")
print(f"   Validation: {len(val_split)} samples (15%)")
print(f"   Test: {len(test_split)} samples (15%)")

# Verify clause type distribution
from collections import Counter

train_types = Counter(s['clause_type'] for s in train_split)
print(f"\n📋 Clause type distribution in training set:")
for clause_type, count in train_types.most_common():
    print(f"   {clause_type}: {count}")


📊 Creating data splits...


NameError: name 'train_data' is not defined

In [ ]:
# Check what's actually in the queries
print("Sample query structure:")
print(json.dumps(list(queries_dict.values())[0], indent=2))
print("\nQuery metadata keys:")
print(list(queries_dict.values())[0].get('metadata', {}).keys())

Sample query structure:
{
  "_id": "New York Governing Law",
  "text": "New York Governing Law",
  "metadata": {
    "category": "Governing Law",
    "parent_query_id": "",
    "type": "cuad1",
    "split": "train"
  }
}

Query metadata keys:
dict_keys(['category', 'parent_query_id', 'type', 'split'])


In [ ]:
# ============================================================================
# CELL 6: Save Converted Data
# ============================================================================

print(f"\n💾 Saving converted data...")
print("=" * 80)

output_dir = "/kaggle/working/acord_data"
os.makedirs(output_dir, exist_ok=True)

# Save each split
for split, data in training_data.items():
    output_path = f"{output_dir}/{split}.json"
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved {split}.json - {len(data)} samples ({os.path.getsize(output_path)/1024/1024:.2f} MB)")

# Save metadata
metadata = {
    'dataset': 'ACORD (Atticus Clause Retrieval Dataset)',
    'source': 'The Atticus Project (Google Drive)',
    'conversion_date': '2025-10-30',
    'min_relevance_score': 1,
    'total_queries': len(queries_dict),
    'total_clauses': len(corpus_dict),
    'splits': {split: len(data) for split, data in training_data.items()},
    'augmentation': {
        'train': 3 if len(training_data.get('train', [])) < 5000 else 1,
        'validation': 1,
        'test': 1
    }
}

with open(f"{output_dir}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ All files saved to: {output_dir}")
print(f"   Files: {', '.join([f'{s}.json' for s in training_data.keys()])}, metadata.json")



💾 Saving converted data...
✅ Saved train.json - 4488 samples (13.19 MB)
✅ Saved test.json - 1726 samples (5.12 MB)

✅ All files saved to: /kaggle/working/acord_data
   Files: train.json, test.json, metadata.json
